In [20]:
import pandas as pd
import pandas_profiling as pp
import plotly.express as px
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier

from pyspark.sql import SparkSession

In [21]:
import json
import time
import schedule
import http.client
import pandas as pd

In [22]:
df = pd.read_csv("./src/data_repo/data/prediction_data.csv")

In [23]:
df.drop('fraud_reported', inplace = True, axis = 1)
df.head()

,months_as_customer,age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,...,bodily_injuries,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year
0,286,43,663190,05-02-1994,IL,100/300,500,1564.43,3000000,477644,...,2,2,YES,34290,3810,3810,26670,Jeep,Grand Cherokee,2013
1,257,44,109392,12-07-2006,OH,100/300,1000,1280.88,0,433981,...,0,1,NO,46980,0,5220,41760,Accura,TL,2002
2,94,26,215278,24-10-2007,IN,100/300,500,722.66,0,433696,...,1,2,YES,36700,3670,7340,25690,Nissan,Pathfinder,2010
3,124,28,674570,08-12-2001,OH,250/500,1000,1235.14,0,443567,...,0,1,?,60200,6020,6020,48160,Volkswagen,Passat,2012
4,141,30,681486,24-03-2007,IN,500/1000,1000,1347.04,0,430665,...,1,2,YES,6480,540,1080,4860,Honda,Civic,1996


In [31]:
df_1 = df[df['policy_number'] > 918516]

In [32]:
df_json = df_1.to_json()

In [33]:
df_json

'{"months_as_customer":{"0":286,"3":124,"4":141,"5":3,"7":130},"age":{"0":43,"3":28,"4":30,"5":38,"7":34},"policy_number":{"0":663190,"3":674570,"4":681486,"5":941851,"7":918516},"policy_bind_date":{"0":"05-02-1994","3":"08-12-2001","4":"24-03-2007","5":"16-07-1991","7":"17-02-2003"},"policy_state":{"0":"IL","3":"OH","4":"IN","5":"OH","7":"OH"},"policy_csl":{"0":"100\\/300","3":"250\\/500","4":"500\\/1000","5":"500\\/1000","7":"250\\/500"},"policy_deductable":{"0":500,"3":1000,"4":1000,"5":1000,"7":500},"policy_annual_premium":{"0":1564.43,"3":1235.14,"4":1347.04,"5":1310.8,"7":1383.49},"umbrella_limit":{"0":3000000,"3":0,"4":0,"5":0,"7":3000000},"insured_zip":{"0":477644,"3":443567,"4":430665,"5":431289,"7":442797},"insured_sex":{"0":"FEMALE","3":"MALE","4":"MALE","5":"FEMALE","7":"FEMALE"},"insured_education_level":{"0":"MD","3":"MD","4":"High School","5":"Masters","7":"Masters"},"insured_occupation":{"0":"prof-specialty","3":"exec-managerial","4":"sales","5":"craft-repair","7":"arme

In [19]:
df_json

'{"months_as_customer":{"0":286,"1":257,"2":94,"3":124,"4":141,"5":3,"6":285,"7":130,"8":458,"9":456},"age":{"0":43,"1":44,"2":26,"3":28,"4":30,"5":38,"6":41,"7":34,"8":62,"9":60},"policy_number":{"0":663190,"1":109392,"2":215278,"3":674570,"4":681486,"5":941851,"6":186934,"7":918516,"8":533940,"9":556080},"policy_bind_date":{"0":"05-02-1994","1":"12-07-2006","2":"24-10-2007","3":"08-12-2001","4":"24-03-2007","5":"16-07-1991","6":"05-01-2014","7":"17-02-2003","8":"18-11-2011","9":"11-11-1996"},"policy_state":{"0":"IL","1":"OH","2":"IN","3":"OH","4":"IN","5":"OH","6":"IL","7":"OH","8":"IL","9":"OH"},"policy_csl":{"0":"100\\/300","1":"100\\/300","2":"100\\/300","3":"250\\/500","4":"500\\/1000","5":"500\\/1000","6":"100\\/300","7":"250\\/500","8":"500\\/1000","9":"250\\/500"},"policy_deductable":{"0":500,"1":1000,"2":500,"3":1000,"4":1000,"5":1000,"6":1000,"7":500,"8":2000,"9":1000},"policy_annual_premium":{"0":1564.43,"1":1280.88,"2":722.66,"3":1235.14,"4":1347.04,"5":1310.8,"6":1436.79,

In [10]:
# Serializing json 
json_object = json.dumps(df_json, indent = 4)
  
# Writing to sample.json
with open("sample.json", "w") as outfile:
    outfile.write(json_object)

In [ ]:
c = http.client.HTTPConnection('127.0.0.1', 8000)
data = pd.read_csv('Data_repository/Data/test.csv')
data = data.to_json()
c.request('POST', '/get_prediction_results', json.dumps(data))
result = c.getresponse().read()
prediction_result = json.loads(result)
prediction_result_df = pd.read_json(prediction_result)
print(prediction_result_df.head())
return prediction_result_df
    

if __name__ == '__main__':
    schedule.every(0).minutes.do(get_prediction_result)
    while True:
        schedule.run_pending()
        time.sleep(1)

In [7]:
data = '{"months_as_customer":{"0":328},"age":{"0":48},"policy_number":{"0":521585},"policy_bind_date":{"0":1413504000000},"policy_state":{"0":"OH"},"policy_csl":{"0":"250\\/500"},"policy_deductable":{"0":1000},"policy_annual_premium":{"0":1406.91},"umbrella_limit":{"0":0},"insured_zip":{"0":466132},"insured_sex":{"0":"MALE"},"insured_education_level":{"0":"MD"},"insured_occupation":{"0":"craft-repair"},"insured_hobbies":{"0":"sleeping"},"insured_relationship":{"0":"husband"},"capital-gains":{"0":53300},"capital-loss":{"0":0},"incident_date":{"0":1422144000000},"incident_type":{"0":"Single Vehicle Collision"},"collision_type":{"0":"Side Collision"},"incident_severity":{"0":"Major Damage"},"authorities_contacted":{"0":"Police"},"incident_state":{"0":"SC"},"incident_city":{"0":"Columbus"},"incident_location":{"0":"9935 4th Drive"},"incident_hour_of_the_day":{"0":5},"number_of_vehicles_involved":{"0":1},"property_damage":{"0":"YES"},"bodily_injuries":{"0":1},"witnesses":{"0":2},"police_report_available":{"0":"YES"},"total_claim_amount":{"0":71610},"injury_claim":{"0":6510},"property_claim":{"0":13020},"vehicle_claim":{"0":52080},"auto_make":{"0":"Saab"},"auto_model":{"0":"92x"},"auto_year":{"0":2004},"fraud_reported":{"0":"Y"}}'


In [8]:
a_json = json.loads(data)


In [9]:
df_pandas = pd.DataFrame.from_dict(a_json)
df_spark = spark.createDataFrame(df_pandas)



In [10]:
import sys
import argparse

def get_args():

    parser = argparse.ArgumentParser(description='Zeiss Task')


    parser.add_argument("--app_name", type=str, default="Zeiss_classification_task")
    parser.add_argument("--data_filename", type=str, default="./data/insurance_claims.csv",
                                    help="data file in csv format")
    parser.add_argument("--columns_to_drop", type=list, 
                        default=['policy_number','policy_bind_date','policy_state','insured_zip','incident_location',
                        'incident_date','auto_make','auto_model','insured_occupation','age', 'total_claim_amount'])
    parser.add_argument("--store_schema", type=bool, default=False)
    parser.add_argument("--schema_path", type=str, default="/src/data/feature_store/")
    parser.add_argument("--columns_to_encode", type=list, 
                        default=['policy_csl', 'insured_sex', 'insured_education_level','insured_hobbies', 'insured_relationship',
                        'incident_type', 'incident_severity','authorities_contacted', 'incident_state', 'incident_city','collision_type'])
    parser.add_argument("--preprocess_hobbies", type=bool, default=True)
    parser.add_argument("--tune_hyper_params", type=bool, default=False)
    parser.add_argument("--best_hyper_params_filepath", type=str, default='/src/data/best_hyper_params')
    parser.add_argument("--model_path", type=str, default='/src/model')
    parser.add_argument("--hyper_params", type=dict, 
                        default={'criterion': 'gini', 
                                    'max_depth': 5, 
                                    'min_samples_leaf': 2, 
                                    'min_samples_split': 2
                                    })
    parser.add_argument("--target", type=str, default='fraud_reported')

    return parser.parse_args()



In [11]:
sys.argv = sys.argv[10:]

In [12]:
args = get_args()

In [14]:
from src.utils.utils import  data_preprocessing

df_processed = data_preprocessing(df_spark, args)



AttributeError: 'OneHotEncoder' object has no attribute 'fit'

In [29]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer.load(os.getcwd() + args.schema_path + '/schemaData/' + 'incident_city')

pyspark.ml.feature.StringIndexer

In [33]:
import pickle
